<a href="https://colab.research.google.com/github/dayaniravi123/Machine-Learning-Projects/blob/main/NLP_Tweeet_Analysis_v_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [19]:
import pandas as pd
import tensorflow as tf
import re
import string
import tensorflow_datasets as tfds
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
## load dataset from google drive folder
train_db_file = '/content/drive/MyDrive/Machine Learning - Tensorflow /Corona NLP/Corona_NLP_train.csv'
test_db_file = '/content/drive/MyDrive/Machine Learning - Tensorflow /Corona NLP/Corona_NLP_test.csv'

train = pd.read_csv(train_db_file, encoding = "ISO-8859-1")
test = pd.read_csv(test_db_file, encoding = "ISO-8859-1")

'''train = tf.data.experimental.make_csv_dataset(
    train_db_file,
    batch_size=32,
    #label_name='survived',
    num_epochs=1,
    ignore_errors=True,)
'''

"train = tf.data.experimental.make_csv_dataset(\n    train_db_file,\n    batch_size=32,\n    #label_name='survived',\n    num_epochs=1,\n    ignore_errors=True,)\n"

In [22]:
## remove the non-relavent columns from dataset

train = train.drop(columns=['UserName','ScreenName','Location','TweetAt'])
test = test.drop(columns=['UserName','ScreenName','Location','TweetAt'])

train_features = train['OriginalTweet']
test_features = test['OriginalTweet']

train['Sentiment'] = train['Sentiment'].astype('category').cat.codes
test['Sentiment'] = test['Sentiment'].astype('category').cat.codes

In [46]:
train.columns

Index(['OriginalTweet', 'Sentiment'], dtype='object')

In [5]:
## save update dataframe in csv file

train.to_csv('train.csv')
test.to_csv('test.csv')

In [6]:
## Load the dataset using tensorflow keras api

train_db_path = '/content/train.csv'
test_db_path = '/content/test.csv'


train = tf.data.experimental.make_csv_dataset(
    train_db_path,
    batch_size=5,
    select_columns = ['OriginalTweet', 'Sentiment'],
    label_name='Sentiment',
    num_epochs=1,
    )

test = tf.data.experimental.make_csv_dataset(
    test_db_path,
    batch_size=5,
    label_name='Sentiment',
    num_epochs=1,
    )




In [7]:
## check the first batch of train dataset

for text, label in train.take(1):
    
    print(text)
    print(text['OriginalTweet'])
    print(label)

OrderedDict([('OriginalTweet', <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'I Think Mike DeWine Is Doing The Right Thing By Limiting Social Interactions Until This #Coronavirus Shit Gets Under Control, But What About Grocery Store Workers... It Has Been Soooo Many People In These Grocery Stores \xc3\x82\xc2\x93PANIC BUYING\xc3\x82\xc2\x94',
       b'9/11 ONLY killed 2,996 people, but #coronavirus kills MORE -  7,514 DEAD!! Restaurants and shops shut down, schools shut down, cities shut down, countries shut down, stock markets crash, markets run out of toilet paper, water, and food with bno end in sight! #Stockpile NOW!!!',
       b'1. Not everyone has the option of working from home- or have the government forgotten that? 2) online shopping now impossible, and shop shelves are empty anyway. If COVID 19 doesn\xc3\x82\xc2\x92t get us, starvation will.. #UKlockdown',
       b"Australian Competition and Consumer Commission (ACCC)'s Scamwatch has received multiple reports of COVID-

In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train = train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test = test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [9]:
## Implement Text Vactorization 

VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation), '')


vectorize_layer = tf.keras.layers.TextVectorization(
    #standardize=custom_standardization,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length = 250)

#train_text = train.map(lambda x, y: x)
vectorize_layer.adapt(train_features)

In [10]:
## Build model

model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [11]:
train_text = train.map(lambda x, y: x)
train_text 
label = train.map(lambda x,y:y)
print(label)

val = train_text.map(lambda x: x['OriginalTweet'])
val
#encoder.adapt(val)

<MapDataset shapes: (None, None), types: tf.int32>


<MapDataset shapes: (None, None), types: tf.string>

In [ ]:
for i in val.take(1):
    print(i)

In [13]:
new_train = train.map(lambda x, y:(x['OriginalTweet'],y))
new_train

<MapDataset shapes: ((None, None), (None, None)), types: (tf.string, tf.int32)>

In [ ]:
history = model.fit(new_train, epochs= 1)

In [23]:
X_train = train['OriginalTweet']
y_train = train['Sentiment']
X_test = test['OriginalTweet']
y_test = test['Sentiment']

In [24]:
model = Pipeline([
                  ('tfidf',TfidfVectorizer()), 
                  ("cb",CatBoostClassifier(random_state=42)
                  )])
model.fit(X_train,y_train)
pred = model.predict(X_test)
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

Learning rate set to 0.095644
0:	learn: 1.5833635	total: 3.44s	remaining: 57m 12s
1:	learn: 1.5625448	total: 6.21s	remaining: 51m 38s
2:	learn: 1.5454373	total: 9s	remaining: 49m 51s
3:	learn: 1.5295913	total: 11.8s	remaining: 48m 47s
4:	learn: 1.5161021	total: 14.6s	remaining: 48m 21s
5:	learn: 1.5057655	total: 17.5s	remaining: 48m 11s
6:	learn: 1.4951654	total: 20.2s	remaining: 47m 41s
7:	learn: 1.4848738	total: 22.9s	remaining: 47m 24s
8:	learn: 1.4765214	total: 25.8s	remaining: 47m 17s
9:	learn: 1.4680530	total: 28.5s	remaining: 47m 1s
10:	learn: 1.4603806	total: 31.3s	remaining: 46m 52s
11:	learn: 1.4538964	total: 34s	remaining: 46m 42s
12:	learn: 1.4472227	total: 36.8s	remaining: 46m 34s
13:	learn: 1.4417225	total: 39.5s	remaining: 46m 23s
14:	learn: 1.4365513	total: 42.3s	remaining: 46m 15s
15:	learn: 1.4314438	total: 45.1s	remaining: 46m 10s
16:	learn: 1.4263787	total: 47.8s	remaining: 46m 4s
17:	learn: 1.4212356	total: 50.6s	remaining: 45m 59s
18:	learn: 1.4167732	total: 53.4s